In [1]:
!pip install transformers peft accelerate bitsandbytes datasets trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 23.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.6/504.6 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 93.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5

In [2]:
from datasets import load_dataset

dataset_path = "/kaggle/input/jsonfiles2/ISO_Format_Dataset.jsonl"  # Update if your filename is different

dataset = load_dataset("json", data_files={"train": dataset_path}, split="train")
dataset = dataset.train_test_split(test_size=0.05)

print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 190
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 10
    })
})


In [ ]:
from huggingface_hub import login
login(token="-----")

In [6]:
from transformers import AutoTokenizer

# Load your tokenizer here
model_name = "mistralai/Mistral-7B-Instruct-v0.1"  # or another model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Ensure pad_token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def format_instruction(example):
    instruction = str(example.get("instruction", ""))
    input_text = str(example.get("input", ""))
    output_text = str(example.get("output", ""))

    prompt = f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:
"""

    input_ids = tokenizer(prompt, truncation=True, max_length=512, padding="max_length")["input_ids"]
    label_ids = tokenizer(output_text, truncation=True, max_length=128, padding="max_length")["input_ids"]

    return {
        "input_ids": input_ids,
        "labels": label_ids
    }

# Assuming `dataset` is a Hugging Face Dataset object
tokenized_dataset = dataset.map(format_instruction, batched=False)


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Map:   0%|          | 0/190 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoModelForCausalLM
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import torch

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)


2025-08-04 21:21:18.053462: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754342478.434767      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754342478.546131      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [8]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Use a very small subset of data
small_train_dataset = tokenized_dataset["train"].select(range(32))  # just 32 samples
small_eval_dataset = tokenized_dataset["test"].select(range(8))     # just 8 samples

training_args = TrainingArguments(
    output_dir="./mistral_datetime_lora_test",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,
    learning_rate=5e-5,                 # higher learning rate for fast convergence
    num_train_epochs=1,                # only 1 epoch
    logging_steps=5,
    save_strategy="no",                # don't save model checkpoints
    fp16=True,                         # faster if you're on Colab with T4/A100/V100
    report_to="none",
)

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator
)

# Train
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:457: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default)

Step,Training Loss
5,4.097800
10,3.424200
15,3.047600


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=16, training_loss=3.491451546549797, metrics={'train_runtime': 65.0073, 'train_samples_per_second': 0.492, 'train_steps_per_second': 0.246, 'total_flos': 699341337526272.0, 'train_loss': 3.491451546549797, 'epoch': 1.0})

In [9]:
model.save_pretrained("./mistral_datetime_lora")
tokenizer.save_pretrained("./mistral_datetime_lora")

('./mistral_datetime_lora/tokenizer_config.json',
 './mistral_datetime_lora/special_tokens_map.json',
 './mistral_datetime_lora/chat_template.jinja',
 './mistral_datetime_lora/tokenizer.model',
 './mistral_datetime_lora/added_tokens.json',
 './mistral_datetime_lora/tokenizer.json')

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model = AutoModelForCausalLM.from_pretrained(
    "./mistral_datetime_lora",
    device_map="auto",  # Will offload intelligently
    offload_folder="./offload",  # Optional but good if memory is tight
    torch_dtype="auto"
)

tokenizer = AutoTokenizer.from_pretrained("./mistral_datetime_lora")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

prompt = """### Instruction:
Extract exact datetime from this expression assuming today is 2025-08-04.

### Input:
yesterday at 8:30

### Response:
"""

output = pipe(prompt, max_new_tokens=64, do_sample=False)
print(output[0]['generated_text'].split("### Response:")[1].strip())

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


2025-08-03 08:30:00
